# PythonによるWebスクレイピング　クローリング

（レポート課題としては，クローリングまでを行う必要はありません．ただし，スクレイピング（クローリング）を行う前に**クローリングの許可／禁止の確認**は重要であること，クローリングを行わないとスクレイピングを行う意味が少ないことから，説明しておきます．）

Googleなどの検索エンジンでは，リンクをたどりながら様々なウェブの情報を収集している．こういった作業をクローリングと呼ぶ．

ここでは，サンプルページ

https://hirotakeyamazoe.jp/lecture/advexp/

に対して，クローリングを行い，スクレイピングを行う例を示す．

## クローリング前のクローリング許可／禁止設定の確認

**クローリングを行う際の注意：**

* ページによってクローリングを禁止している場合があるため，クローリングが許可されているかを確認する必要がある．

* 許可されていた場合でも，サーバへの負荷軽減のため，アクセスごとに**最低でも１秒以上程度の待ち時間**を設定する必要がある（ページごとに秒数が指定されている場合もある）．

クローリングを行う前に，クローリングが禁止されていないかを確認する必要がある．クローリングの許可／禁止の設定方法としては，

* robots.txtを用いる方法

* metaタグを用いる方法

など，いくつかの方法がある．

ここでは，robots.txtに着目し，これを解釈する方法として```urllib.robotparser```を利用する方法を紹介する．

### ```urllib.robotparser```でrobots.txtを解釈する

robot.txtを解析するための準備として，ライブラリの準備と，robots.txtの読み込みを行う．

一般に，robots.txtはウェブサイトのルートディレクトリに置かれることが多い．

（例：https://hirotakeyamazoe.jp/lecture/advexp/ の場合，https://hirotakeyamazoe.jp/robots.txt に置かれていることが多い）



In [1]:
import urllib.robotparser

robots_url = "https://hirotakeyamazoe.jp/robots.txt"

rp = urllib.robotparser.RobotFileParser()
rp.set_url(robots_url)
rp.read()

次に，チェックしたページのアクセス許可を確認する．

サンプルページ```https://hirotakeyamazoe.jp/lecture/advexp/```は，Googleなどにクローリングされないために拒否と設定しており，```False```が返される
（実際には，クローリングして構わない）．

In [2]:
rp.can_fetch("*", "https://hirotakeyamazoe.jp/lecture/advexp/")

False

ここで，```rp.can_fetch()```の第1引数はユーザエージェントであり，ワイルドカード（すべてのユーザエージェント）と設定している．

ユーザエージェントとは，Webページへのアクセスに用いているソフト（ブラウザ）のことで，ブラウザ名・OS名によって表示される情報が変化する場合がある（例えば，パソコンかスマートフォンかによって表示が変化するなど）．

```https://hirotakeyamazoe.jp/lecture/```は，クローリングを許可しているため，```True```が返される．

In [3]:
rp.can_fetch("*", "https://hirotakeyamazoe.jp/lecture/")

True

## リンク先のページをたどって情報収集（クローリング）

### htmlの取得からBeautiful Soupへの入力まで（復習）

In [4]:
# Requests, BeautifulSoupの読み込み
import requests
from bs4 import BeautifulSoup

# 読み込むURLの指定
url = 'https://hirotakeyamazoe.jp/lecture/advexp/'

# 指定のURLへアクセス
result = requests.get(url)

# 取得結果から文字コードを推定して設定
# この処理を入れないと、結果の出力で文字化けする）
result.encoding = result.apparent_encoding

# Beautiful Soupを用いて解析
soup = BeautifulSoup(result.text, 'html5lib')

# h1要素の取得と結果の表示（例）
h1 = soup.find('h1')
print(h1.text)

電子情報工学特別実験用サンプルページ


### リンク先URLリストの取得
リンクは```<a href=..> </a>```と表現されるため，```a```タグを取得すればよい．

```a```タグには，リンクテキストとリンク先情報が含まれる．

*   リンクテキスト：h1などと同じように，```a.text```で取得
*   リンク先URL：```a['href']```で取得

In [5]:
# aタグの取得
a_list = soup.find_all('a')

# リンクテキスト，リンク先URLの表示
for a in a_list:
  print(a.text, a['href'])

電気工学コース ./denki.html
電子情報工学コース ./denshi.html
機械工学コース ./kikai.html
材料工学コース ./zairyo.html
応用化学コース ./ouyou.html
化学工学コース ./kagaku.html


### 絶対アドレスへの変換

ただし，HTMLファイル内で指定されるURLは，現在のページを基準とした相対アドレスであるため，絶対アドレスに変換する必要がある．

上の例では，「電気工学コース」のURLは，```./denshi.html```と表示されているが，
実際のURLは```https://hirotakeyamazoe.jp/lecture/advexp/denki.html```である．

```urllib```ライブラリの関数```urllib.parse.urljoin(url, a['href'])```を利用することで，絶対アドレスに変換できる．

In [6]:
# urllibの読み込み
import urllib

# aタグの取得
a_list = soup.find_all('a')

for a in a_list:
  # 絶対アドレスへの変換
  a_url = urllib.parse.urljoin(url, a['href'])

  # リンクテキスト，リンク先URL（絶対アドレス）の表示
  print(a.text, a_url)


電気工学コース https://hirotakeyamazoe.jp/lecture/advexp/denki.html
電子情報工学コース https://hirotakeyamazoe.jp/lecture/advexp/denshi.html
機械工学コース https://hirotakeyamazoe.jp/lecture/advexp/kikai.html
材料工学コース https://hirotakeyamazoe.jp/lecture/advexp/zairyo.html
応用化学コース https://hirotakeyamazoe.jp/lecture/advexp/ouyou.html
化学工学コース https://hirotakeyamazoe.jp/lecture/advexp/kagaku.html


### リンク先URLのページから情報の取得

絶対アドレスの取得後は，htmlの取得とBeautiful Soupによる解析を繰り返して行っていけばよい．

例えば，リンク先の全てのコースのページの説明文を取得し，CSVファイルに保存する場合には，以下のようなコードとなる．

In [7]:
# Requests, BeautifulSoupの読み込み
import requests
from bs4 import BeautifulSoup

# urllibの読み込み
import urllib

# time（時間待ち用），csv（CSV保存用）の読み込み
import time
import csv

# 最初に読み込むURLの指定
url = 'https://hirotakeyamazoe.jp/lecture/advexp/'

# 指定のURLへアクセス
result = requests.get(url)

# 取得結果から文字コードを推定して設定
# この処理を入れないと、結果の出力で文字化けする）
result.encoding = result.apparent_encoding

# Beautiful Soupを用いて解析
soup = BeautifulSoup(result.text, 'html5lib')

# 元ページからリンク先リストを取得
a_list = soup.find_all('a')

c_res_list = []
for a in a_list:
  # 絶対アドレスへの変換
  a_url = urllib.parse.urljoin(url, a['href'])

  # コースの名前（リンクテキスト）を取得
  c_name = a.text

  # リンク先のURLへアクセス
  c_res = requests.get(a_url)

  # 取得結果から文字コードを推定して設定
  c_res.encoding = c_res.apparent_encoding

  # Beautiful Soupを用いて解析
  c_soup = BeautifulSoup(c_res.text, 'html5lib')

  # リンク先ページの説明文（p要素）の取得
  c_desc = c_soup.p.text

  print(c_name, a_url, c_desc)
  c_res_list.append([c_name, a_url, c_desc])

  # 重要：待ち時間（１秒以上必要）．複数ページに対してアクセスする際には必要
  time.sleep(1)

# CSVファイルへの書き込み
with open('course.csv', 'wt', encoding='sjis', newline='') as fp:
  csv.writer(fp).writerows(c_res_list)

電気工学コース https://hirotakeyamazoe.jp/lecture/advexp/denki.html 電気系工学の専門基礎力をしっかりと身に付けた上で、電気エネルギーの発生から利用までの電力システムに関する知識と技術、エレクトリニクスの基礎を支える電子物性から電気・電子材料に関する知識、これらの電気電子材料を活かしたデバイスに関する技術を習得する。
電子情報工学コース https://hirotakeyamazoe.jp/lecture/advexp/denshi.html 電子材料・デバイスから回路、高周波、通信、信号処理システムまでの幅広い領域に対して基礎知識を学ぶと同時に、最先端技術に関する専門知識や実践的技術を学びます。
機械工学コース https://hirotakeyamazoe.jp/lecture/advexp/kikai.html 人にやさしい21世紀のヒューマン・エンジニアリングの担い手として、他の分野の専門技術と有機的に結びつきながら次世代のニーズに柔軟に応えられる機械技術者・研究者を育成することを目標にしています。
材料工学コース https://hirotakeyamazoe.jp/lecture/advexp/zairyo.html 創造性・独創性・国際性豊かな21世紀型の材料工学関する技術者・研究者の養成を目指す。
応用化学コース https://hirotakeyamazoe.jp/lecture/advexp/ouyou.html 応用化学コースでは、“化学=Chemistry”が関わるエネルギー、情報、医薬など、すべての分野に関連した化学を学びます。
化学工学コース https://hirotakeyamazoe.jp/lecture/advexp/kagaku.html 化学工学コースでは新規物質を探索する「マテリアルサイエンス(物質科学)」と、製造プロセスを最適化する「ケミカルエンジニアリング(化学工学)」の二つの学問体系を融合させた教育プログラムを提供します。
